# Identificação

**Assunto:** Modelagem

**Tutor:** Manoel Veríssimo dos Santos Neto e Matheus Patusco

## 1- Objetivos de Aprendizagem
Neste notebook, iremos:
1. Recuperar o dataset processado em CSV.
2. Comparar múltiplos algoritmos de machine learning.
3. Armazenar e versionar no MLflow o modelo com melhor desempenho e menor custo computacional.

### 1.1- Bibliotecas Necessárias
    

In [ ]:
# Manipulação e visualização de dados
import pandas as pd
import seaborn as sns
import time

# Bibliotecas para aprendizado de máquina
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score

# MLflow para gerenciamento de experimentos
import mlflow
import mlflow.sklearn

# Supressão de avisos
import warnings
warnings.filterwarnings("ignore")

## 2- Recuperando o Dataset do MLflow

In [ ]:
# Especificar o caminho do artefato no MLflow
artifact_path = "../02-dados/dados_processados.csv"

# Carregar o dataset processado
dados = pd.read_csv(artifact_path)
dados.head()

## 3- Dividindo os Dados para Treinamento e Teste

In [ ]:
# Separando as features (X) e o target (y)
X = dados.drop(columns=["Survived", "Name", "Ticket", "Cabin", "Embarked"], errors='ignore')  # Substitua 'Survived' pelo nome da coluna alvo, se necessário
y = dados["Survived"]  # Substitua 'Survived' pelo nome da coluna alvo, se necessário

# Divisão dos dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Conjunto de treinamento: {X_train.shape}")
print(f"Conjunto de teste: {X_test.shape}")

## 4- Comparando Algoritmos de Machine Learning

In [ ]:
# Lista de modelos para comparar
modelos = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42)
}

resultados = []

# Avaliar cada modelo
for nome, modelo in modelos.items():
    inicio = time.time()
    modelo.fit(X_train, y_train)  # Treinamento
    fim = time.time()

    # Previsões
    y_pred = modelo.predict(X_test)

    # Métricas
    acuracia = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="weighted")
    tempo_treino = fim - inicio

    # Registrar no MLflow
    with mlflow.start_run(run_name=nome):
        mlflow.log_param("Modelo", nome)
        mlflow.log_metric("Acurácia", acuracia)
        mlflow.log_metric("F1-Score", f1)
        mlflow.log_metric("Tempo de Treinamento", tempo_treino)
        mlflow.sklearn.log_model(modelo, "modelo")

    # Armazenar resultados
    resultados.append({
        "Modelo": nome,
        "Acurácia": acuracia,
        "F1-Score": f1,
        "Tempo de Treinamento (s)": tempo_treino
    })
    print(f"Modelo {nome} treinado e registrado no MLflow.")

## 5- Resultados da Comparação

In [ ]:
# Criar um DataFrame com os resultados
df_resultados = pd.DataFrame(resultados)
df_resultados.sort_values(by=["Acurácia", "Tempo de Treinamento (s)"], ascending=[False, True], inplace=True)
print("Resultados da Comparação:")
print(df_resultados)

# Exibir o modelo com melhor desempenho
melhor_modelo = df_resultados.iloc[0]
print(f"Melhor Modelo: {melhor_modelo['Modelo']}")

## 6- Armazenando o Melhor Modelo no MLflow

In [ ]:
# Recuperar o modelo com melhor desempenho
nome_melhor_modelo = melhor_modelo["Modelo"]
modelo_final = modelos[nome_melhor_modelo]

# Armazenar o modelo final no MLflow
with mlflow.start_run(run_name="Melhor Modelo"):
    mlflow.log_param("Modelo", nome_melhor_modelo)
    mlflow.log_metric("Acurácia", melhor_modelo["Acurácia"])
    mlflow.log_metric("F1-Score", melhor_modelo["F1-Score"])
    mlflow.log_metric("Tempo de Treinamento", melhor_modelo["Tempo de Treinamento (s)"])
    mlflow.sklearn.log_model(modelo_final, "melhor_modelo")
print(f"Melhor modelo ({nome_melhor_modelo}) armazenado com sucesso no MLflow.")

## 7- Exercícios


1.   Verificar a documentação dos modelos RandomForestClassifier, LogisticRegression, KNeighborsClassifier, GradientBoostingClassifier e altere ou inclua algum parâmetro dos modelos e compare os resultados com o baseline executado nesse notebook.
2.   Busque algum outro dataset no Kaggle para um problema de regressão e faça um novo treino. Lembre de modificar as métricas, ex.: MSE.
3.   Execute o MLFlow de maneira que se parar o container os dados não sejam perdidos, podendo salvar os dados ou no SQLite (default) ou outro banco de dados da sua escolha.

**Importante:**

*   Todas as alterações devem ser registradas no Mlflow (subir em container) para que seja possível realizar comparações entre os experimentos.

## Conclusão


Este notebook demonstrou como comparar múltiplos algoritmos de machine learning, avaliar seus desempenhos e armazenar o melhor modelo no MLflow.
O MLflow foi utilizado para rastrear e versionar os experimentos e os modelos de forma eficaz.
    